In [6]:
# resource https://learn.deeplearning.ai/langchain/lesson/2/models,-prompts-and-parsers

In [1]:
# !pip install --upgrade pip
# !pip install openai
# !pip install --upgrade langchain

In [2]:
from langchain.chat_models import ChatOpenAI

In [5]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')

In [6]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)
# IMPORTANT: delete output if you preview it, as it contains the api key

### Prompt template

In [39]:
from langchain.prompts import ChatPromptTemplate

schema = """ {'DIM_ADDRESS': [{'column_name': 'ADDRESS_CITY_NAME', 'data_type': 'TEXT'}, {'column_name': 'ADDRESS_COUNTRY_NAME', 'data_type': 'TEXT'}, {'column_name': 'ADDRESS_ID', 'data_type': 'NUMBER'}, {'column_name': 'ADDRESS_PK', 'data_type': 'TEXT'}, {'column_name': 'ADDRESS_STATE_NAME', 'data_type': 'TEXT'}], 'DIM_CREDIT_CARD': [{'column_name': 'CREDITCARD_ID', 'data_type': 'NUMBER'}, {'column_name': 'CREDITCARD_PK', 'data_type': 'TEXT'}, {'column_name': 'CREDITCARD_TYPE', 'data_type': 'TEXT'}], 'DIM_CUSTOMER': [{'column_name': 'BUSINESS_ENTITY_ID', 'data_type': 'NUMBER'}, {'column_name': 'CUSTOMER_FULL_NAME', 'data_type': 'TEXT'}, {'column_name': 'CUSTOMER_ID', 'data_type': 'NUMBER'}, {'column_name': 'CUSTOMER_PK', 'data_type': 'TEXT'}, {'column_name': 'CUSTOMER_STORE_BUSINESS_ENTITY_ID', 'data_type': 'NUMBER'}, {'column_name': 'CUSTOMER_STORE_NAME', 'data_type': 'TEXT'}], 'DIM_DATE': [{'column_name': 'DATE_DAY', 'data_type': 'DATE'}, {'column_name': 'DATE_PK', 'data_type': 'TEXT'}, {'column_name': 'DAY_OF_MONTH', 'data_type': 'NUMBER'}, {'column_name': 'DAY_OF_WEEK', 'data_type': 'NUMBER'}, {'column_name': 'DAY_OF_WEEK_NAME', 'data_type': 'TEXT'}, {'column_name': 'DAY_OF_YEAR', 'data_type': 'NUMBER'}, {'column_name': 'NEXT_DATE_DAY', 'data_type': 'DATE'}, {'column_name': 'PRIOR_DATE_DAY', 'data_type': 'DATE'}, {'column_name': 'PRIOR_YEAR_DATE_DAY', 'data_type': 'DATE'}, {'column_name': 'PRIOR_YEAR_OVER_YEAR_DATE_DAY', 'data_type': 'DATE'}], 'DIM_ORDER_STATUS': [{'column_name': 'ORDER_STATUS', 'data_type': 'NUMBER'}, {'column_name': 'ORDER_STATUS_NAME', 'data_type': 'TEXT'}, {'column_name': 'ORDER_STATUS_PK', 'data_type': 'TEXT'}], 'DIM_PRODUCT': [{'column_name': 'PRODUCT_CATEGORY_NAME', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_CLASS', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_COLOR', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_ID', 'data_type': 'NUMBER'}, {'column_name': 'PRODUCT_NAME', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_NUMBER', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_PK', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_SUBCATEGORY_NAME', 'data_type': 'TEXT'}], 'FCT_SALES': [{'column_name': 'CREDITCARD_FK', 'data_type': 'TEXT'}, {'column_name': 'CUSTOMER_FK', 'data_type': 'TEXT'}, {'column_name': 'ORDER_DATE_FK', 'data_type': 'TEXT'}, {'column_name': 'ORDER_STATUS_FK', 'data_type': 'TEXT'}, {'column_name': 'PRODUCT_FK', 'data_type': 'TEXT'}, {'column_name': 'SALE_ORDER_DETAIL_ID', 'data_type': 'NUMBER'}, {'column_name': 'SALE_ORDER_ID', 'data_type': 'NUMBER'}, {'column_name': 'SALE_ORDER_QUANTITY', 'data_type': 'NUMBER'}, {'column_name': 'SALE_PK', 'data_type': 'TEXT'}, {'column_name': 'SALE_REVENUE', 'data_type': 'NUMBER'}, {'column_name': 'SALE_UNIT_PRICE', 'data_type': 'NUMBER'}, {'column_name': 'SHIP_ADDRESS_FK', 'data_type': 'TEXT'}]} """

table_name = "DIM_ADDRESS"
# column_name = "ADDRESS_CITY_NAME"
column_name = "ADDRESS_ID"

format_instructions = """\

Format the output as JSON with the following keys:
* dimension_name: the name of the dimension. If a dimension made from this column is not appropriate then this field should be "null" (enclosed in double quotes).
* dimension_type: the type of the dimension. If the column is not a dimension, then this field should be "null" (enclosed in double quotes)
* measure_1_name: the name of the first measure. If a measure made from this column is not appropriate then this field should be "null" (enclosed in double quotes).
* measure_1_type: the type of the first measure. If a measure made from this column is not appropriate then this field should be "null" (enclosed in double quotes).
* measure_2_name: the name of the second measure. If a second measure made from this column is not appropriate then this field should be "null" (enclosed in double quotes).
* measure_2_type: the type of the second measure. If a second measure made from this column is not appropriate then this field should be "null" (enclosed in double quotes).

"""

cube_objects_from_schema_template = """\
The dictionary below details a snowflake schema for a list of tables and their columns. \
I want you to, for the table {table_name} (the key of the dictionary) and colum_name: {column_name}, decide whether \
the column is a dimension or a measure (or both). Follow these rules \
    1. you can make between 0 and 2 measures of a column
    2. you can make between 0 and 1 dimensions of a column
    3. dimension names should match the column name exactly
    4. measure names should be the column name with a relevant prefix in lowercase e.g. "sum_" or "avg_"

schema: {schema}

{format_instructions}
```
"""

backup:

prompt

The dictionary below details a snowflake schema for a list of tables and their columns. \
I want you to, for each column, decide whether: \
    1. The column is a dimension or a measure (or both). Follow these rules \
        A: you can make as many measures of a column as you deem appropriate \
        B: you can make at most one domension of a column \
        C: you need to determine the type of the dimension or measure \
    3. The column is a primary key or a foreign key \

definitions

table_name: the name of the table

column_name: the name of the column

dimension_name: the name of the dimension. If the column is not a dimension, then this field should be null. A column can correspond to at most one dimension.

dimension_type: the type of the dimension. If the column is not a dimension, then this field should be null

primary key: whether the column underlying a dimension is the primary key of the table. There can only be one per table

measure_name: the name of the measure. A column can correspond to multiple measures

measure_type: the type of the measure

dictionary

{'table_name': [ // the key is a string; the name of the table. The value is a list of dictionaries; each dictionary represents a column of the table
    { 'column_name': [ // the key is a string; the name of the column. The value is a list of dictionaries; each dictionary represents either the dimension or list of measures of the column
        { 'dimensions: [ // the value is a list of dictionaries; each dictionary represents a dimension of the column. But there should at most be one dictionary here, because a column can correspond to at most one dimension. If the column is not a dimension, then this list shpuld be empty
            {
                'dimension_name': // string; the name of the dimension
                ,'dimension_type': // string; the type of the dimension
                ,'primary_key': // boolean; whether the column underlying this dimension is the primary key. Answer True if yes, False if not or unknown. There can only be one primary key per table.
            }
        ] }
        ,{ 'measures': [ // the value is a list of dictionaries; each dictionary represents a measure of the column. If the column is not a measure, then this list should be empty
            {
                'measure_name': // string; the name of the measure
                ,'measure_type': // string; the type of the measure
            }
            ,{
                <more measures of the column>
            }
        ] }
    ] }
}

In [40]:
prompt = ChatPromptTemplate.from_template(template=cube_objects_from_schema_template)
prompt

ChatPromptTemplate(input_variables=['column_name', 'table_name', 'format_instructions', 'schema'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['column_name', 'format_instructions', 'schema', 'table_name'], output_parser=None, partial_variables={}, template='The dictionary below details a snowflake schema for a list of tables and their columns. I want you to, for the table {table_name} (the key of the dictionary) and colum_name: {column_name}, decide whether the column is a dimension or a measure (or both). Follow these rules     1. you can make between 0 and 2 measures of a column\n    2. you can make between 0 and 1 dimensions of a column\n    3. dimension names should match the column name exactly\n    4. measure names should be the column name with a relevant prefix in lowercase e.g. "sum_" or "avg_"\n\nschema: {schema}\n\n{format_instructions}\n```\n', template_format='f-string', validate_template=True), addi

In [41]:
messages = prompt.format_messages(
    table_name=table_name,
    column_name=column_name,
    schema=schema, 
    format_instructions=format_instructions,
)
print(messages[0].content)

The dictionary below details a snowflake schema for a list of tables and their columns. I want you to, for the table DIM_ADDRESS (the key of the dictionary) and colum_name: ADDRESS_ID, decide whether the column is a dimension or a measure (or both). Follow these rules     1. you can make between 0 and 2 measures of a column
    2. you can make between 0 and 1 dimensions of a column
    3. dimension names should match the column name exactly
    4. measure names should be the column name with a relevant prefix in lowercase e.g. "sum_" or "avg_"

schema:  {'DIM_ADDRESS': [{'column_name': 'ADDRESS_CITY_NAME', 'data_type': 'TEXT'}, {'column_name': 'ADDRESS_COUNTRY_NAME', 'data_type': 'TEXT'}, {'column_name': 'ADDRESS_ID', 'data_type': 'NUMBER'}, {'column_name': 'ADDRESS_PK', 'data_type': 'TEXT'}, {'column_name': 'ADDRESS_STATE_NAME', 'data_type': 'TEXT'}], 'DIM_CREDIT_CARD': [{'column_name': 'CREDITCARD_ID', 'data_type': 'NUMBER'}, {'column_name': 'CREDITCARD_PK', 'data_type': 'TEXT'}, {'c

In [42]:
response = chat(messages)

In [43]:
print(response.content)

{
    "dimension_name": "ADDRESS_ID",
    "dimension_type": "DIMENSION",
    "measure_1_name": "null",
    "measure_1_type": "null",
    "measure_2_name": "null",
    "measure_2_type": "null"
}


# todos:

fix
* figure out how to avoid exposing api keys when committing to github
* provide list of measure and dimension types as input
* specify how to name dimensions and measures

change
* enable the use of `output_parser`
    * make a separate call for each column, so the schema is known?
    * how to deal with multiple measures? Limit to 3?

add 
 * foreign key: whether the column is a foreign key of the table

In [12]:
output_dict = output_parser.parse(response.content)

NameError: name 'output_parser' is not defined

In [ ]:
output_dict

In [ ]:
type(output_dict)